Varios LGBM y regresion lineal y media 093 - ensemble

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

print("Librerías importadas correctamente")
print("Ensemble de 3 modelos: LGBM + Regresión Lineal + Media 12 093")

Librerías importadas correctamente
Ensemble de 3 modelos: LGBM + Regresión Lineal + Media 12 093


In [4]:
# Cargar los cinco archivos de predicciones
print("Cargando archivos de predicciones...")

# Cargar predicciones de LGBM (sem_1_promedio)
pred_lgbm = pd.read_csv('data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto_sem_1_promedio.csv')
print(f"LGBM - Shape: {pred_lgbm.shape}")
print("LGBM - Primeras filas:")
print(pred_lgbm.head(3))

print("\n" + "="*50 + "\n")

# Cargar predicciones de LGBM2 (producto)
pred_lgbm2 = pd.read_csv('data/pred_lgbm_v3_FE_02_v3_linear_tree_opt_03_36lags_producto.csv')
print(f"LGBM2 - Shape: {pred_lgbm2.shape}")
print("LGBM2 - Primeras filas:")
print(pred_lgbm2.head(3))

print("\n" + "="*50 + "\n")

# Cargar predicciones de LGBM3
pred_lgbm3 = pd.read_csv('data/pred_lgbm_v3_FE_03_linear_tree_opt_03_36lags_producto.csv')
print(f"LGBM3 - Shape: {pred_lgbm3.shape}")
print("LGBM3 - Primeras filas:")
print(pred_lgbm3.head(3))

print("\n" + "="*50 + "\n")

# Cargar predicciones de Regresión Lineal
pred_regresion = pd.read_csv('data/pred_regresion_lineal01.csv')
print(f"Regresión Lineal - Shape: {pred_regresion.shape}")
print("Regresión Lineal - Primeras filas:")
print(pred_regresion.head(3))

print("\n" + "="*50 + "\n")

# Cargar predicciones de Media 12 093
pred_media = pd.read_csv('data/pred_medias_12_093.csv')
print(f"Media 12 093 - Shape: {pred_media.shape}")
print("Media 12 093 - Primeras filas:")
print(pred_media.head(3))

# Resumen de shapes de todos los archivos cargados
print("\nResumen de shapes de todos los archivos cargados:")
print(f"LGBM: {pred_lgbm.shape}")
print(f"LGBM2: {pred_lgbm2.shape}")
print(f"LGBM3: {pred_lgbm3.shape}")
print(f"Regresión Lineal: {pred_regresion.shape}")
print(f"Media 12 093: {pred_media.shape}")

Cargando archivos de predicciones...
LGBM - Shape: (780, 2)
LGBM - Primeras filas:
   product_id           tn
0       20001  1282.319435
1       20002  1009.995347
2       20003   667.117963


LGBM2 - Shape: (780, 2)
LGBM2 - Primeras filas:
   product_id           tn
0       20001  1300.846482
1       20002  1007.214398
2       20003   679.601498


LGBM3 - Shape: (780, 2)
LGBM3 - Primeras filas:
   product_id           tn
0       20001  1352.622594
1       20002  1179.363975
2       20003   696.798612


Regresión Lineal - Shape: (780, 2)
Regresión Lineal - Primeras filas:
   product_id           tn
0       20001  1162.707525
1       20002  1183.640604
2       20003   684.763931


Media 12 093 - Shape: (780, 2)
Media 12 093 - Primeras filas:
   product_id           tn
0       20001  1352.901430
1       20002  1093.156542
2       20003   730.028059

Resumen de shapes de todos los archivos cargados:
LGBM: (780, 2)
LGBM2: (780, 2)
LGBM3: (780, 2)
Regresión Lineal: (780, 2)
Media 12 093: (7

In [5]:
# Verificar que los tres archivos tienen los mismos productos
print("Verificando consistencia de datos...")
print(f"Productos únicos en LGBM: {pred_lgbm['product_id'].nunique()}")
print(f"Productos únicos en Regresión Lineal: {pred_regresion['product_id'].nunique()}")
print(f"Productos únicos en Media 12 093: {pred_media['product_id'].nunique()}")

# Verificar si tienen exactamente los mismos product_ids
productos_lgbm = set(pred_lgbm['product_id'])
productos_regresion = set(pred_regresion['product_id'])
productos_media = set(pred_media['product_id'])

# Verificar intersecciones
productos_comunes = productos_lgbm & productos_regresion & productos_media
print(f"\nProductos en común entre los 3 modelos: {len(productos_comunes)}")

if len(productos_comunes) == len(productos_lgbm) == len(productos_regresion) == len(productos_media):
    print("✅ Los tres archivos tienen exactamente los mismos productos")
else:
    print("⚠️ Los archivos NO tienen exactamente los mismos productos")
    
    # Verificar diferencias
    solo_lgbm = productos_lgbm - productos_regresion - productos_media
    solo_regresion = productos_regresion - productos_lgbm - productos_media
    solo_media = productos_media - productos_lgbm - productos_regresion
    
    if solo_lgbm:
        print(f"Solo en LGBM: {len(solo_lgbm)} productos")
    if solo_regresion:
        print(f"Solo en Regresión: {len(solo_regresion)} productos")
    if solo_media:
        print(f"Solo en Media: {len(solo_media)} productos")

print(f"\nUsaremos los {len(productos_comunes)} productos comunes para el ensemble")

Verificando consistencia de datos...
Productos únicos en LGBM: 780
Productos únicos en Regresión Lineal: 780
Productos únicos en Media 12 093: 780

Productos en común entre los 3 modelos: 780
✅ Los tres archivos tienen exactamente los mismos productos

Usaremos los 780 productos comunes para el ensemble


In [6]:
# Crear ensemble promediando las cinco predicciones
print("Creando ensemble de 5 modelos...")

# Renombrar las columnas tn para distinguir los modelos
pred_lgbm_renamed = pred_lgbm.rename(columns={'tn': 'tn_lgbm'})
pred_lgbm2_renamed = pred_lgbm2.rename(columns={'tn': 'tn_lgbm2'})
pred_lgbm3_renamed = pred_lgbm3.rename(columns={'tn': 'tn_lgbm3'})
pred_regresion_renamed = pred_regresion.rename(columns={'tn': 'tn_regresion'})
pred_media_renamed = pred_media.rename(columns={'tn': 'tn_media'})

# Hacer merge de los cinco dataframes por product_id
print("Haciendo merge de los cinco modelos...")
ensemble_step1 = pd.merge(pred_lgbm_renamed, pred_lgbm2_renamed, on='product_id', how='inner')
ensemble_step2 = pd.merge(ensemble_step1, pred_lgbm3_renamed, on='product_id', how='inner')
ensemble_step3 = pd.merge(ensemble_step2, pred_regresion_renamed, on='product_id', how='inner')
ensemble_data = pd.merge(ensemble_step3, pred_media_renamed, on='product_id', how='inner')

print(f"Datos después del merge: {ensemble_data.shape}")
print("\nPrimeras filas del merge:")
print(ensemble_data.head())

# Calcular el promedio de las cinco predicciones
ensemble_data['tn'] = (ensemble_data['tn_lgbm'] + ensemble_data['tn_lgbm2'] + ensemble_data['tn_lgbm3'] + ensemble_data['tn_regresion'] + ensemble_data['tn_media']) / 5

print(f"\nEstadísticas del ensemble:")
print(f"Promedio de tn: {ensemble_data['tn'].mean():.2f}")
print(f"Mediana de tn: {ensemble_data['tn'].median():.2f}")
print(f"Std de tn: {ensemble_data['tn'].std():.2f}")

# Mostrar comparación de los primeros productos
print(f"\nComparación de predicciones para los primeros 5 productos:")
comparison = ensemble_data[['product_id', 'tn_lgbm', 'tn_lgbm2', 'tn_lgbm3', 'tn_regresion', 'tn_media', 'tn']].head()
print(comparison)

Creando ensemble de 5 modelos...
Haciendo merge de los cinco modelos...
Datos después del merge: (780, 6)

Primeras filas del merge:
   product_id      tn_lgbm     tn_lgbm2     tn_lgbm3  tn_regresion  \
0       20001  1282.319435  1300.846482  1352.622594   1162.707525   
1       20002  1009.995347  1007.214398  1179.363975   1183.640604   
2       20003   667.117963   679.601498   696.798612    684.763931   
3       20004   528.675142   557.937422   539.329044    580.484961   
4       20005   558.755442   556.942803   564.775686    563.560780   

      tn_media  
0  1352.901430  
1  1093.156542  
2   730.028059  
3   583.310255  
4   621.491197  

Estadísticas del ensemble:
Promedio de tn: 35.29
Mediana de tn: 8.93
Std de tn: 90.75

Comparación de predicciones para los primeros 5 productos:
   product_id      tn_lgbm     tn_lgbm2     tn_lgbm3  tn_regresion  \
0       20001  1282.319435  1300.846482  1352.622594   1162.707525   
1       20002  1009.995347  1007.214398  1179.363975   11

In [7]:
# Crear el archivo final con solo product_id y tn promediado
print("Preparando datos finales para guardar...")

# Seleccionar solo las columnas necesarias para el archivo final
resultado_final = ensemble_data[['product_id', 'tn']].copy()

# Verificar que no hay valores nulos
print(f"Valores nulos en el resultado final: {resultado_final.isnull().sum().sum()}")
print(f"Shape del resultado final: {resultado_final.shape}")

# Ordenar por product_id para consistencia
resultado_final = resultado_final.sort_values('product_id').reset_index(drop=True)

print("\nPrimeras 10 filas del resultado final:")
print(resultado_final.head(10))

print(f"\nÚltimas 5 filas del resultado final:")
print(resultado_final.tail())

Preparando datos finales para guardar...
Valores nulos en el resultado final: 0
Shape del resultado final: (780, 2)

Primeras 10 filas del resultado final:
   product_id           tn
0       20001  1290.279493
1       20002  1094.674173
2       20003   691.662012
3       20004   557.947365
4       20005   573.105181
5       20006   426.057987
6       20007   371.156518
7       20008   366.370875
8       20009   434.459964
9       20010   371.338181

Últimas 5 filas del resultado final:
     product_id        tn
775       21263  0.941802
776       21265  0.922707
777       21266  0.899068
778       21267  1.017463
779       21276  0.985521


In [8]:
# Guardar el archivo final
archivo_salida = "data/pred_ensemble05_lgbm_rl_media12_09302.csv"

print(f"Guardando resultado en: {archivo_salida}")

# Crear directorio data si no existe
import os
os.makedirs("data", exist_ok=True)

# Guardar el archivo
resultado_final.to_csv(archivo_salida, index=False)

print(f"✅ Archivo guardado exitosamente: {archivo_salida}")
print(f"Total de productos en el ensemble: {len(resultado_final)}")

# Verificar que el archivo se guardó correctamente
if os.path.exists(archivo_salida):
    tamaño_archivo = os.path.getsize(archivo_salida)
    print(f"Tamaño del archivo: {tamaño_archivo} bytes")
    
    # Leer las primeras líneas del archivo guardado para verificar
    verificacion = pd.read_csv(archivo_salida, nrows=5)
    print(f"\nVerificación - Primeras 5 líneas del archivo guardado:")
    print(verificacion)
else:
    print("⚠️ Error: No se pudo crear el archivo")

Guardando resultado en: data/pred_ensemble05_lgbm_rl_media12_09302.csv
✅ Archivo guardado exitosamente: data/pred_ensemble05_lgbm_rl_media12_09302.csv
Total de productos en el ensemble: 780
Tamaño del archivo: 19886 bytes

Verificación - Primeras 5 líneas del archivo guardado:
   product_id           tn
0       20001  1290.279493
1       20002  1094.674173
2       20003   691.662012
3       20004   557.947365
4       20005   573.105181


In [ ]:
# Análisis estadístico final del ensemble
print("RESUMEN DEL ENSEMBLE DE 5 MODELOS")
print("="*60)

# Estadísticas descriptivas de cada modelo
print("Estadísticas de LGBM:")
print(f"  Promedio: {ensemble_data['tn_lgbm'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_lgbm'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_lgbm'].std():.2f}")
print(f"  Min:      {ensemble_data['tn_lgbm'].min():.2f}")
print(f"  Max:      {ensemble_data['tn_lgbm'].max():.2f}")

print("\nEstadísticas de LGBM2:")
print(f"  Promedio: {ensemble_data['tn_lgbm2'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_lgbm2'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_lgbm2'].std():.2f}")
print(f"  Min:      {ensemble_data['tn_lgbm2'].min():.2f}")
print(f"  Max:      {ensemble_data['tn_lgbm2'].max():.2f}")

print("\nEstadísticas de LGBM3:")
print(f"  Promedio: {ensemble_data['tn_lgbm3'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_lgbm3'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_lgbm3'].std():.2f}")
print(f"  Min:      {ensemble_data['tn_lgbm3'].min():.2f}")
print(f"  Max:      {ensemble_data['tn_lgbm3'].max():.2f}")

print("\nEstadísticas de Regresión Lineal:")
print(f"  Promedio: {ensemble_data['tn_regresion'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_regresion'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_regresion'].std():.2f}")
print(f"  Min:      {ensemble_data['tn_regresion'].min():.2f}")
print(f"  Max:      {ensemble_data['tn_regresion'].max():.2f}")

print("\nEstadísticas de Media 12 093:")
print(f"  Promedio: {ensemble_data['tn_media'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn_media'].median():.2f}")
print(f"  Std:      {ensemble_data['tn_media'].std():.2f}")
print(f"  Min:      {ensemble_data['tn_media'].min():.2f}")
print(f"  Max:      {ensemble_data['tn_media'].max():.2f}")

print("\nEstadísticas del Ensemble (Promedio de 5 modelos):")
print(f"  Promedio: {ensemble_data['tn'].mean():.2f}")
print(f"  Mediana:  {ensemble_data['tn'].median():.2f}")
print(f"  Std:      {ensemble_data['tn'].std():.2f}")
print(f"  Min:      {ensemble_data['tn'].min():.2f}")
print(f"  Max:      {ensemble_data['tn'].max():.2f}")

# Correlaciones entre modelos
print("\nMatriz de correlaciones:")
correlaciones = ensemble_data[['tn_lgbm', 'tn_lgbm2', 'tn_lgbm3', 'tn_regresion', 'tn_media']].corr()
print(correlaciones.round(4))

print(f"\n✅ Ensemble de 5 modelos completado exitosamente!")
print(f"Archivo final: {archivo_salida}")
print(f"Total de productos: {len(resultado_final)}")
print(f"Método: Promedio simple de 5 modelos")

RESUMEN DEL ENSEMBLE DE 3 MODELOS
Estadísticas de LGBM:
  Promedio: 34.87
  Mediana:  8.85
  Std:      87.64
  Min:      0.59
  Max:      1282.32

Estadísticas de Regresión Lineal:
  Promedio: 35.67
  Mediana:  9.04
  Std:      92.24
  Min:      0.23
  Max:      1183.64

Estadísticas de Media 12 093:
  Promedio: 36.54
  Mediana:  8.86
  Std:      95.81
  Min:      0.02
  Max:      1352.90

Estadísticas del Ensemble (Promedio de 3 modelos):
  Promedio: 35.69
  Mediana:  8.88
  Std:      91.70
  Min:      0.40
  Max:      1265.98

Matriz de correlaciones:
              tn_lgbm  tn_regresion  tn_media
tn_lgbm        1.0000        0.9912    0.9957
tn_regresion   0.9912        1.0000    0.9942
tn_media       0.9957        0.9942    1.0000

✅ Ensemble de 3 modelos completado exitosamente!
Archivo final: data/pred_lgbm_rl_media12_09302.csv
Total de productos: 780
Método: Promedio simple de LGBM + Regresión Lineal + Media 12 093
